# Step-by-Step 모델 학습 (Denoising: DnCNN + Deconvolution: U-Net)


## 1. 환경 설정 및 데이터 준비


In [ ]:
from google.colab import drive
import os
import shutil
import time
from pathlib import Path

# --- 1. 환경 설정 ---
print("--- 1. Setting up environment ---")
drive.mount('/content/drive')

# TODO: 사용자님의 Google Drive 프로젝트 경로에 맞게 수정해주세요.
PROJECT_ROOT = '/content/drive/MyDrive/Data Scientist/Project/Week5/week5'
os.chdir(PROJECT_ROOT)
print(f"Changed directory to: {os.getcwd()}")

# --- Git 동기화 ---
print("\n--- Syncing with GitHub Repository ---")
!git pull origin main
print("--- Git sync complete ---\n")

%pip install loguru tqdm --quiet
print("Installed libraries.")

# --- 2. 데이터셋 준비 (로컬 런타임으로 복사) ---
print("\n--- 2. Preparing dataset ---")
DRIVE_DATASET_ROOT = Path('./dataset')
LOCAL_DATASET_ROOT = Path('/content/dataset')
folders_to_copy = ['train', 'val']

print(f"Starting optimized data copy to {LOCAL_DATASET_ROOT}...")
start_time = time.time()

if LOCAL_DATASET_ROOT.exists():
    print("Local dataset folder already exists. Skipping copy.")
else:
    LOCAL_DATASET_ROOT.mkdir(parents=True, exist_ok=True)
    for folder in folders_to_copy:
        source = DRIVE_DATASET_ROOT / folder
        destination = LOCAL_DATASET_ROOT / folder
        if source.exists():
            print(f"Copying {source} to {destination}...")
            shutil.copytree(source, destination)
            print(f"Finished copying {folder}.")
        else:
            print(f"Source folder {source} not found, skipping.")

end_time = time.time()
print(f"Data preparation finished in {end_time - start_time:.2f} seconds.")


## 2. [1단계] Denoising 모델 (DnCNN) 학습

원본 이미지에 **노이즈만 추가**하여 노이즈 제거를 위한 DnCNN 모델을 학습시킵니다.


In [ ]:
!python "code_denoising/train_controlled.py" \
    --model_type "dncnn" \
    --run_dir "logs_sbs_denoising_dncnn" \
    --augmentation_mode "noise_only"


## 3. [2단계] Deconvolution 모델 (U-Net) 학습

원본 이미지에 **컨볼루션(흐림 효과)만 추가**하여 흐림 효과 제거를 위한 U-Net 모델을 학습시킵니다.


In [ ]:
!python "code_denoising/train_controlled.py" \
    --model_type "unet" \
    --run_dir "logs_sbs_deconv_unet" \
    --augmentation_mode "conv_only"
